[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [1]:
import numpy as np
import pandas as pd
import math
from common import *

norm = np.linalg.norm
pd.set_option('display.max_colwidth', -1)


### [Wrapper za brojanje fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

In [2]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        f.called = 0
        try:
            return fn(f, *args, **kwargs)
        finally:
            wrapper.calls_to_function = f.called
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije cilja

In [117]:
pt = lambda *x: np.array(x, dtype=float)
dim = lambda pt: len(pt)

@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [4]:
@counts
def golden_section(f, a, b, e=1e-6, k=0.5*(math.sqrt(5) + 1)):
    hi = b - (b - a) / k
    lo = a + (b - a) / k
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while abs(b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - (b - a) / k
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + (b - a) / k
            f_hi = f_lo
            f_lo = f(lo)
    
    return (a + b) / 2

In [5]:
@counted 
def test_gs_fn(x): 
    return (x - 4.99)**2

print(golden_section(test_gs_fn, 0, 5))
print(golden_section.calls_to_function)

4.989999842904119
35


### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [6]:
@counts
def unimodal(f, pt, h=1):
    l, r = pt - h, pt + h
    m = pt
    step = 1
    
    fm = f(pt)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while True:
            l, m, fm = m, r, fr
            r += h * step
            fr = f(r)
            step *= 2
            if fm <= fr:
                return l, r
    else:
        while True:
            r, m, fm = m, l, fl
            l -= h * step
            fl = f(l)
            step *= 2
            if fm <= fl:
                return l, r
    

### Coordinate descent
TODO uncopy

In [15]:
@counts
def coordinate_descent(f, x0, eps=1e-6):
    x = np.array(x0)
    x_prev = x + 2 * eps
    while norm(x_prev - x) > eps:
        x_prev = np.copy(x)
        for i in range(len(x)):
            def fi(xt):
                xp = np.copy(x)
                xp[i] = xt
                return f(xp)
            l, r = unimodal(fi, 0.1, x[i])
            x[i] = golden_section(fi, l, r, eps)
    return x

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [16]:
class InvalidAlgorithmSetupException(Exception):
    def __init__(self, message):
        super(InvalidAlgorithmSetupException, self).__init__(message)
    
    def raiseIf(condition, message = "Invalid algorithm setup"):
        if condition:
            raise InvalidAlgorithmSetupException(message)

In [17]:
@counts
def nelder_mead(
    f, x0, step=1, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6, maxIter=1000):
    InvalidAlgorithmSetupException.raiseIf(alpha <= 0)
    InvalidAlgorithmSetupException.raiseIf(gamma <= 1)
    InvalidAlgorithmSetupException.raiseIf(sigma <= 0 or sigma > 0.5)
    
    x = simplex_pts(x0, step)
    l, h = 0, -1
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    expand = lambda xc, xr: xc + gamma * (xr - xc)
    contract = lambda xc, xh: xc - beta * (xc - xh)
    shrink = lambda xl, xi: xl + sigma * (xi - xl)
    
    i = 0
    while True:
        i += 1
        
        x = sorted(x, key=f)
        xc = np.mean(x[:h], axis=0)
        
        if norm(x[h] - xc) <= epsilon or i >= maxIter:
            break
        
        xr = reflect(xc, x[h])
        
        fxr = f(xr)
        fxl = f(x[l])
        
        if fxr < fxl:
            xe = expand(xc, x[h])
            x[h] = xe if f(xe) < fxl else xr
        else:
            if all(fxr > f(xj) for xj in x[:h]):
                if fxr < f(x[h]):
                    x[h] = xr
                    
                xk = contract(xc, x[h])
                if f(xk) < f(x[h]):
                    x[h] = xk
                else:
                    shrink_xi = lambda xi: shrink(x[l], xi)
                    x = lmap(shrink_xi, x)
            else:
                x[h] = xr
            
    return x[0]

def simplex_pts(x0, step):
    ret = [x0]
    for i in range(len(x0)):
        xp = np.copy(x0)
        xp[i] += step
        ret.append(xp)
    return np.array(ret)
        

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [52]:
np.copy([1]) / 2

array([ 0.5])

In [124]:
@counts
def hooke_jeeves(f, x0, epsilon=1e-6, dx=0.5):
    dx = dx * np.ones_like(x0)
    epsilon = epsilon * np.ones_like(x0)

    xp = np.copy(x0)
    xb = np.copy(x0)

    while all(dx > epsilon):
        xn = explore(f, xp, dx)
        if f(xn) < f(xb):
            xp = 2 * xn - xb
            xb = np.copy(xn)
        else:
            dx /= 2.0
            xp = np.copy(xb)
        
    return xp

def explore(f, xp, dx):
    x = np.array(xp, dtype=float)
    P = f(x)
    for i in range(len(xp)):
        x[i] = float(x[i]) + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

In [126]:
hooke_jeeves(f4, pt(5, 14))

array([ 9.5,  9.5])

In [127]:
hooke_jeeves.calls_to_function

157

# Zadaci
## 1.
Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [21]:
data = []

for x0 in [ pt(10), pt(20), pt(30) ]:
    for f in [ hooke_jeeves, nelder_mead, coordinate_descent ]:
        result = f(f3, x0)
        data.append([f.__name__, x0, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,[10],76,[10]
1,nelder_mead,[10],207,[1.0]
2,coordinate_descent,[10],77,[1]
3,hooke_jeeves,[20],76,[20]
4,nelder_mead,[20],257,[1.0]
5,coordinate_descent,[20],86,[0]
6,hooke_jeeves,[30],76,[30]
7,nelder_mead,[30],307,[1.0]
8,coordinate_descent,[30],80,[1]


## 2.
Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za
svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata?

In [22]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]

data = []
for search_f in [hooke_jeeves, nelder_mead]:
    for f, x0 in zip(fs, x0s):
        result = search_f(f, x0)
        data.append(
            [search_f.__name__, x0, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,"[-1.9, 2.0]",694,"[1.00000152588, 1.0000038147]"
1,hooke_jeeves,"[0.1, 0.3]",305,"[3.99999961853, 2.00000076294]"
2,hooke_jeeves,"[0, 0, 0, 0, 0]",228,"[0, 0, 0, 0, 0]"
3,hooke_jeeves,"[5.1, 1.1]",172,"[3.1, 3.1]"
4,nelder_mead,"[-1.9, 2.0]",3562,"[0.999999763998, 0.999999487937]"
5,nelder_mead,"[0.1, 0.3]",493,"[3.99999995768, 1.99999972735]"
6,nelder_mead,"[0, 0, 0, 0, 0]",2774,"[0.999999846924, 1.99999971796, 3.00000003086, 4.00000002405, 4.99999947917]"
7,nelder_mead,"[5.1, 1.1]",712,"[-8.3331812619e-07, 1.51895016504e-07]"


## 3.
Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate!

In [23]:
hooke_jeeves(f4, pt(5, 5))

array([5, 5])

In [24]:
nelder_mead(f4, pt(5, 5))

array([ -8.84859430e-08,  -3.86806220e-07])

## 4.
Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

## 5.
Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

In [25]:
ncalls = 20

data = []
for i in range(ncalls):
    random = np.random.randint(-50, 50 + 1, size=2)
    result = hooke_jeeves(f6, pt(*random))
    data.append([hooke_jeeves.__name__, 
                 random,
                 hooke_jeeves.calls_to_function, 
                 result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,"[-13, -15]",149,"[-11, -15]"
1,hooke_jeeves,"[13, -11]",146,"[13, -9]"
2,hooke_jeeves,"[6, 22]",114,"[6, 22]"
3,hooke_jeeves,"[6, 34]",114,"[6, 34]"
4,hooke_jeeves,"[-8, -24]",159,"[-7, -24]"
5,hooke_jeeves,"[-8, 35]",138,"[1, 35]"
6,hooke_jeeves,"[26, -27]",133,"[26, -27]"
7,hooke_jeeves,"[16, -9]",133,"[16, -9]"
8,hooke_jeeves,"[48, 33]",114,"[48, 33]"
9,hooke_jeeves,"[-44, 15]",144,"[-41, 15]"


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)